# RedisAI

In [76]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from ml2rt import save_sklearn, load_model
from redisai import Client


In [77]:
# RedisAI Client
redisai_client = Client(host='evc.re.kr', port=6379)

In [78]:
# model output path
MODEL_OUTPUT_PATH = '../test_model/iris.onnx'
model_onnx = load_model(MODEL_OUTPUT_PATH)
model_onnx


b'\x08\x08\x12\x08skl2onnx\x1a\x061.11.2"\x07ai.onnx(\x002\x00:\xfa\x04\n\x93\x02\n\x08features\x12\x05label\x12\x12probability_tensor\x1a\x10LinearClassifier"\x10LinearClassifier*\x1b\n\x10classlabels_ints@\x00@\x01@\x02\xa0\x01\x07*M\n\x0ccoefficients=#\xd1\xc4\xbe=\xaaAq?=\xfa\x17\x17\xc0=tf\x80\xbf=\xf9\x03\xfe>=\xed\xeeh\xbe=\x0c\xee5\xbe=\xce\x1a5\xbf=W\xcb\xe4\xbd=\xef\x057\xbf=\xdbv"@=\xdb\xf3\xda?\xa0\x01\x06*\x1e\n\nintercepts=\xe5\xae\rA=\xb8-\xc2?=\x9c\xf4%\xc1\xa0\x01\x06*\x12\n\x0bmulti_class\x18\x01\xa0\x01\x02*\x1c\n\x0epost_transform"\x07SOFTMAX\xa0\x01\x03:\nai.onnx.ml\n.\n\x05label\x12\x0coutput_label\x1a\x04Cast"\x04Cast*\t\n\x02to\x18\x07\xa0\x01\x02:\x00\nV\n\x12probability_tensor\x12\rprobabilities\x1a\nNormalizer"\nNormalizer*\r\n\x04norm"\x02L1\xa0\x01\x03:\nai.onnx.ml\n^\n\rprobabilities\x12\x12output_probability\x1a\x06ZipMap"\x06ZipMap*\x1d\n\x12classlabels_int64s@\x00@\x01@\x02\xa0\x01\x07:\nai.onnx.ml\x12 c1fc8eb7409d4677b95523efcb907620Z\x16\n\x08features

In [79]:
redisai_client.modelset('iris-clf', 'onnx', 'cpu', model_onnx)
print(f">> '{model_name}'' model is saved to RedisAI 🟢.")

    

>> 'iris-clf'' model is saved to RedisAI 🟢.


/tmp/ipykernel_3832452/331947039.py:1: DeprecationWarning: Call to deprecated method modelset. (Use modelstore instead) -- Deprecated since version 1.2.0.
  redisai_client.modelset('iris-clf', 'onnx', 'cpu', model_onnx)


- https://www.kaggle.com/code/pramod7/starter-iris-dataset-with-pytorch

In [80]:
import torch as tr
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as f
from sklearn.model_selection import train_test_split
import pandas as pd

#load the iris data
df=pd.read_csv('../input/iris.csv')
# read the columns
print (df.columns)

# separate them into features and labels
x=df.drop('target',axis=1).values
y=df['target'].values

# split them into train and test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

#convert x_train and x_test into float tensors.  y_train and y_test into int
x_train=tr.FloatTensor(x_train)
x_test=tr.FloatTensor(x_test)
y_train=tr.LongTensor(y_train)
y_test=tr.LongTensor(y_test)


# build NN tensors model using python class objects, three hidden layers and fully connected by using linear model
class model (nn.Module):
    def __init__(self,in_feat=4,h1=12,h2=12,h3=9,out_feat=3):
        super().__init__()
        self.layr1=nn.Linear(in_feat,h1)
        self.layr2=nn.Linear(h1,h2)
        self.layr3=nn.Linear(h2,h3)
        self.outlayr=nn.Linear(h3,out_feat)
        
    def forward(self,inpts):
        out1=f.relu(self.layr1(inpts)) #output of 1st layer
        out2=f.relu(self.layr2(out1))
        out3=f.relu(self.layr3(out2))
        outpt=self.outlayr(out3)
        return outpt

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')


In [81]:
# data is ready, model is ready. Now lets prepare it to be trained with Epochs
epochs=150
loss_list=[]
algo=model()

#creating criterion for finding the loss values between predicted values and y train values
criterion = nn.CrossEntropyLoss()
optimizer = tr.optim.Adam(algo.parameters(), lr=0.1)

for e in range(epochs):
    e=e+1
    training_pred=algo.forward(x_train)
    loss=criterion(training_pred,y_train)
    loss_list.append(loss)
    
    #below is to print the loss values for each epochs
    if e%5==1:
        print(f'epoch: {e:2}  loss: {loss.item():10.4f}')        
    
    #reset the optimizer    
    optimizer.zero_grad() # reset grad
    loss.backward()
    optimizer.step()

epoch:  1  loss:     1.1844
epoch:  6  loss:     1.0958
epoch: 11  loss:     1.0634
epoch: 16  loss:     0.7430
epoch: 21  loss:     0.4722
epoch: 26  loss:     0.4481
epoch: 31  loss:     0.4032
epoch: 36  loss:     0.3861
epoch: 41  loss:     0.3503
epoch: 46  loss:     0.2826
epoch: 51  loss:     0.1843
epoch: 56  loss:     0.1211
epoch: 61  loss:     0.0775
epoch: 66  loss:     0.0900
epoch: 71  loss:     0.0736
epoch: 76  loss:     0.0774
epoch: 81  loss:     0.0768
epoch: 86  loss:     0.0687
epoch: 91  loss:     0.0617
epoch: 96  loss:     0.0574
epoch: 101  loss:     0.0565
epoch: 106  loss:     0.0561
epoch: 111  loss:     0.0550
epoch: 116  loss:     0.0542
epoch: 121  loss:     0.0537
epoch: 126  loss:     0.0532
epoch: 131  loss:     0.0528
epoch: 136  loss:     0.0524
epoch: 141  loss:     0.0520
epoch: 146  loss:     0.0516


In [82]:
#plot the loss list
#plt.plot(range(epochs), loss_list)
#plt.ylabel('Loss')
#plt.xlabel('epoch')

In [83]:
# we are testing the model to check the loss against test data, Here we do not require back propagation
with tr.no_grad():
    y_pred_loss=algo.forward(x_test)
    val_loss=criterion(y_pred_loss, y_test)
    print (f'{val_loss:.4f}')
    
#let us validate the predicated classes against the actual class
correct=0
with tr.no_grad():
    for i,cls in enumerate (x_test):
        y_pred=algo.forward(cls)
        print(f'{i+1:2}  {str(y_pred):38} {y_test[i]}')
        if y_pred.argmax().item()==y_test[i]:
            correct=correct+1
    print(correct)

0.0319
 1  tensor([-22.6962,  -0.9826,  11.3952]) 2
 2  tensor([ 13.3147,  -0.6264, -22.8869]) 0
 3  tensor([-1.9696,  3.5913, -1.7328])    1
 4  tensor([-1.9696,  3.5913, -1.7328])    1
 5  tensor([ 10.9170,   0.0352, -19.5683]) 0
 6  tensor([-1.9696,  3.5913, -1.7328])    1
 7  tensor([-16.8521,   0.3071,   7.6936]) 2
 8  tensor([-1.9696,  3.5913, -1.7328])    1
 9  tensor([-1.9696,  3.5913, -1.7328])    1
10  tensor([-1.9696,  3.5913, -1.7328])    1
11  tensor([-1.9696,  3.5913, -1.7328])    1
12  tensor([-12.3881,   1.2922,   4.8662]) 2
13  tensor([ 12.0256,  -0.2707, -21.1028]) 0
14  tensor([-3.1092,  3.3398, -1.0110])    1
15  tensor([-1.9696,  3.5913, -1.7328])    1
16  tensor([-1.9696,  3.5913, -1.7328])    1
17  tensor([-17.5459,   0.1540,   8.1330]) 2
18  tensor([-20.9847,  -0.6049,  10.3112]) 2
19  tensor([ 11.8498,  -0.2222, -20.8594]) 0
20  tensor([ 10.6279,   0.1150, -19.1682]) 0
21  tensor([-19.0413,  -0.1760,   9.0803]) 2
22  tensor([ 13.3024,  -0.6231, -22.8700]) 0
23 

In [84]:
from ml2rt import save_sklearn, load_model

    
fpath_model = './tmp/iris.pt'

tr.save(algo.state_dict(), fpath_model)
new_model = model()
new_model.load_state_dict(tr.load(fpath_model))
new_model.eval()

with tr.no_grad():
    y = new_model.forward(x_test)
    loss = criterion(y, y_test)
print(f'{loss:.8f}')


0.03190468


In [85]:
x_test

tensor([[7.6000, 3.0000, 6.6000, 2.1000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [5.7000, 2.9000, 4.2000, 1.3000],
        [5.8000, 2.6000, 4.0000, 1.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [6.7000, 3.0000, 5.2000, 2.3000],
        [5.2000, 2.7000, 3.9000, 1.4000],
        [5.1000, 2.5000, 3.0000, 1.1000],
        [6.0000, 2.2000, 4.0000, 1.0000],
        [5.7000, 2.8000, 4.1000, 1.3000],
        [6.1000, 2.6000, 5.6000, 1.4000],
        [5.1000, 3.5000, 1.4000, 0.3000],
        [6.1000, 2.9000, 4.7000, 1.4000],
        [6.7000, 3.1000, 4.7000, 1.5000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [5.7000, 2.5000, 5.0000, 2.0000],
        [7.7000, 3.0000, 6.1000, 2.3000],
        [5.0000, 3.3000, 1.4000, 0.2000],
        [4.4000, 3.0000, 1.3000, 0.2000],
        [6.9000, 3.2000, 5.7000, 2.3000],
        [5.4000, 3.9000, 1.3000, 0.4000],
        [7.4000, 2.8000, 6.1000, 1.9000],
        [6.3000, 2.7000, 4.9000, 1